In [26]:
if 'google.colab' in str(get_ipython()):
    GITHUB_TOKEN = ""
    !rm -rf anti_money_laundering
    !git clone https://{GITHUB_TOKEN}@github.com/FedericoBruzzone/anti_money_laundering.git
    !mv anti_money_laundering/.* .
    !mv anti_money_laundering/* .
    !rm -rf ./anti_money_laundering

In [2]:
# import os
# from dotenv import load_dotenv

# load_dotenv()

# os.environ['KAGGLE_USERNAME'] = os.getenv("KAGGLE_USER")
# os.environ['KAGGLE_KEY'] = os.getenv('KAGGLE_KEY')
# !cd datasets
# !kaggle datasets download -d --unzip {os.getenv('KAGGLE_DATASET_LINK')}
# !cd ..

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()

import time
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from src.utils.kaggle_config            import setup_kaggle
from src.utils.kaggle_config            import download_dataset

from src.utils.datasets_handler         import (get_train_and_test,
                                                get_X_and_Y,
                                                print_dataset,
                                                label_encoder,
                                                split_timestamp)
from src.utils.performance_measures     import calculate_performances
from src.utils.dataset_sampling_methods import (oversampling,
                                                undersampling,
                                                bootstrap_sampling)

from src.utils.print_utils              import (printLBlue, printGreen)

from src.decision_tree.decision_tree    import CustomDecisionTree
from src.decision_tree.ID3              import DecisionTreeID3
from src.decision_tree.C45              import DecisionTreeC45
from src.decision_tree.entropy_type     import EntropyType
from src.decision_tree.criterion_type   import CriterionType

from IPython.display import Image, display

from src.utils.spark_config import get_spark_session
from pyspark import TaskContext
import pandas as pd

VERBOSE = int(os.getenv('VERBOSE'))
VIEW = os.getenv('VIEW')

setup_kaggle()
print("Downloading dataset...") 
download_dataset("iammustafatz/diabetes-prediction-dataset")
download_dataset("ealtman2019/ibm-transactions-for-anti-money-laundering-aml")
print("Done.")

hi_small_trans = "HI-Small_Trans.csv"
diabetes = "diabetes_prediction_dataset.csv"

Done.


In [29]:
index_tree = 0

def create_trees(partition_elements, verbose=False):
    list_series = []
    for element in partition_elements:
        series_tmp = pd.Series(element.asDict())
        list_series.append(series_tmp)

    part_df = pd.DataFrame(list_series, columns=COLUMNS_NAME)
    X_train, y_train = get_X_and_Y(part_df, verbose=VERBOSE)
    X_train, _ = label_encoder(X_train, ['Timestamp', 'Account', 'Account.1', 'Receiving Currency', 'Payment Currency', 'Payment Format'])

    if index_tree == 0:
        decision_tree: DecisionTreeID3 = DecisionTreeID3(max_depth=8,
                                                        num_thresholds_numerical_attr=2,
                                                        VERBOSE=False)
    elif index_tree == 1:
        decision_tree: CustomDecisionTree = CustomDecisionTree(criterion=EntropyType.SHANNON, 
                                                            type_criterion=CriterionType.BEST, 
                                                            max_depth=20, 
                                                            min_samples_split=100,
                                                            num_thresholds_numerical_attr=2,
                                                            VERBOSE=False)
    elif index_tree == 2:
        decision_tree: DecisionTreeC45 = DecisionTreeC45(max_depth=4,
                                                        min_samples_split=100,
                                                        VERBOSE=False)
    

    decision_tree.fit(X_train, y_train)

    if verbose:
        ctx = TaskContext()
        decision_tree.create_dot_files(filename="tree" + str(ctx.partitionId()),
                                       generate_png=True,
                                       view="default-viewer")
    yield decision_tree

def predict_trees(new_line):
    def wrap(tree):
        prediction = tree.predict(new_line)
        return prediction
    return wrap

def predict_trees_all(X_test):
    def wrap(tree):
        predictions = tree.predict_test_no_gen(X_test)
        return predictions
    return wrap

In [30]:
name = "AntiMoneyLaundering"

spark = get_spark_session(name, VERBOSE)

df_train, df_test = get_train_and_test(hi_small_trans, verbose=VERBOSE)

df_train_p = df_train[df_train["Is Laundering"] == 1]
df_train_n = df_train[df_train["Is Laundering"] == 0]
df_train_n = df_train_n.sample(frac=0.1, random_state=2)
df_train = pd.concat([df_train_p, df_train_n])

df_train = oversampling(df_train, VERBOSE=False)
df_train = bootstrap_sampling(df_train)
print("len(df_train):", len(df_train))

COLUMNS_NAME: list = df_train.columns.tolist()
X_train, y_train = get_X_and_Y(df_train, verbose=VERBOSE)
X_test, y_test = get_X_and_Y(df_test, verbose=VERBOSE)

# X_train, _ = label_encoder(X_train, ['Timestamp', 'Account', 'Account.1', 'Receiving Currency', 'Payment Currency', 'Payment Format'])
X_test, _ = label_encoder(X_test, ['Timestamp', 'Account', 'Account.1', 'Receiving Currency', 'Payment Currency', 'Payment Format'])

len(df_train): 670558


In [31]:
df = spark.createDataFrame(df_train)

print("Printing spark dataframe...")
df.show()

rdd = df.rdd

Printing spark dataframe...
+----------------+---------+---------+-------+---------+---------------+------------------+-----------+-----------------+--------------+-------------+
|       Timestamp|From Bank|  Account|To Bank|Account.1|Amount Received|Receiving Currency|Amount Paid| Payment Currency|Payment Format|Is Laundering|
+----------------+---------+---------+-------+---------+---------------+------------------+-----------+-----------------+--------------+-------------+
|2022/09/02 13:32|     3698|801997D20|  11974|801998AE0|        4109.34|         US Dollar|    4109.34|        US Dollar|           ACH|            1|
|2022/09/06 08:59|    20486|807EF78D0|  19477|80750D550|      127652.54|              Yuan|  127652.54|             Yuan|           ACH|            1|
|2022/09/07 05:35|   228101|80A48A710|  23289|808839F70|       12934.66|          UK Pound|   12934.66|         UK Pound|           ACH|            1|
|2022/09/08 09:41|      222|811D80C30|    121|8000E1590|        51

In [32]:
def map_to_column_value_pairs(row):
    return [(i, row[i]) for i in range(len(row))]

def count_values(a, b):
    return a + b

In [36]:
# ID3
index_tree = 0
predictions = rdd.mapPartitions(create_trees, False) \
                 .map(predict_trees_all(X_test)) \
                 .flatMap(map_to_column_value_pairs) \
                 .map(lambda x: (x, 1)) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0][0], [(x[0][1], x[1])])) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0], max(x[1], key=lambda item: item[1]))) \
                 .map(lambda x: x[1][0]) \
                 .collect()
calculate_performances(predictions, y_test, verbose=True)

ERROR:root:KeyboardInterrupt while sending command.                (0 + 0) / 12]
Traceback (most recent call last):
  File "/home/fcb/Documents/anti-money-laundering/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/fcb/Documents/anti-money-laundering/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Custom
index_tree = 1
predictions = rdd.mapPartitions(create_trees, False) \
                 .map(predict_trees_all(X_test)) \
                 .flatMap(map_to_column_value_pairs) \
                 .map(lambda x: (x, 1)) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0][0], [(x[0][1], x[1])])) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0], max(x[1], key=lambda item: item[1]))) \
                 .map(lambda x: x[1][0]) \
                 .collect()
calculate_performances(predictions, y_test, verbose=True)


ERROR:root:KeyboardInterrupt while sending command.               (0 + 12) / 12]
Traceback (most recent call last):
  File "/home/fcb/Documents/anti-money-laundering/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/fcb/Documents/anti-money-laundering/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# C4.5
index_tree = 2
predictions = rdd.mapPartitions(create_trees, False) \
                 .map(predict_trees_all(X_test)) \
                 .flatMap(map_to_column_value_pairs) \
                 .map(lambda x: (x, 1)) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0][0], [(x[0][1], x[1])])) \
                 .reduceByKey(count_values) \
                 .map(lambda x: (x[0], max(x[1], key=lambda item: item[1]))) \
                 .map(lambda x: x[1][0]) \
                 .collect()
calculate_performances(predictions, y_test, verbose=True)

In [9]:
spark.stop()